In [7]:
import glob
import os
import numpy as np
import pandas as pd
import datetime as dt
from urllib.request import urlretrieve

In [8]:
urls ={}
urls['2013_14']='https://aql.datapress.com/leeds/dataset/payments-to-the-3rd-sector/vcse.csv'
urls['2014_15']='https://aql.datapress.com/leeds/dataset/payments-to-the-3rd-sector/2015-10-26T15:11:28/vcse.csv'
urls['2015_16']='https://aql.datapress.com/leeds/dataset/payments-to-the-3rd-sector/2016-08-22T15:32:09/2015%2016.csv'

# Excluded 2012_13 which uses a different template

In [9]:
# Download missing files
for year, url in urls.items():
    fname = year+'.csv'
    if not os.path.isfile(fname):
        urlretrieve(url, fname) 

In [10]:
#Load headers
xl = pd.ExcelFile('Leeds City Council - Payments to the 3rd sector - Column Headers.xlsx')
dfHead = xl.parse('headers', index_col='File')
dfHead

,Funding Org:Name,OrganisationCode,Planned Dates:Start Date,Classifications:0:Title,Recipient Org:Name,Recipient Org:Identifier,Award Date,Identifier,Amount Awarded,Amount Disbursed,IrrecoverableVAT,Grant Programme:Title,Classifications:1:Title
File,,,,,,,,,,,,,
2015_16,OrganisationName,OrganisationCode,EffectiveDate,Directorate Service,SupplierName,Local Supplier Internal Reference,Payment Date,TransactionNumber,NetAmount,NetAmount2,Irrecoverable VAT,PurposeofSpend,Council Procurement Category
2014_15,OrganisationName,OrganisationCode,EffectiveDate,DirectorateService,SupplierName,LocalSupplierInternalReference,PaymentDate,TransactionNumber,NetAmount,NetAmount2,IrrecoverableVAT,PurposeOfSpend,CouncilProcurementCategoryName
2013_14,OrganisationName,OrganisationCode,EffectiveDate,DirectorateService,SupplierName,LocalSupplierInternalReference,PaymentDate,TransactionNumber,NetAmount,NetAmount2,IrrecoverableVAT,PurposeOfSpend,CouncilProcurementCategoryName


In [11]:
if not os.path.isdir('out'):
    os.mkdir('out')
    
for fname in glob.glob('*.csv'):
    iname = fname.split('.')[0]
    df = pd.read_csv(fname, encoding='cp1252', dtype=object)
    
    df['NetAmount'] = pd.to_numeric(df['NetAmount'].str.replace(',', ''), errors='coerce')    
    df['NetAmount2'] = df['NetAmount']    
    sourceCols = dfHead.ix[[iname]].values.tolist()[0]
    newCols = dfHead.columns.values.tolist()
    df = df[sourceCols]
    df.rename(columns=dict(zip(sourceCols, newCols)), inplace=True)
    df = df[df.Identifier.notnull()]
    df['Identifier'] = '360G-LCC-'+df['Identifier'].astype(str)
    df['Recipient Org:Identifier'] = '360G-LCC-'+df['Recipient Org:Identifier'].astype(str)
    df['Funding Org:Identifier'] = 'GB-LAE-LDS'
    df['Award Date'] = pd.to_datetime(df['Award Date'], errors='coerce')
    df['Award Date'] = df['Award Date'].dt.strftime('%Y-%m-%d').astype(str)
    if 'Planned Dates:Start Date' in newCols:
        df['Planned Dates:Start Date'] = pd.to_datetime(df['Planned Dates:Start Date'], errors='coerce')
        df['Planned Dates:Start Date'] = df['Planned Dates:Start Date'].dt.strftime('%Y-%m-%d').astype(str)
    df['Last modified'] = dt.date.strftime(dt.datetime.now(),'%Y-%m-%dT%H:%M:%SZ')
    df['Data Source'] =  'https://datamillnorth.org/dataset/payments-to-the-3rd-sector'
    df['Currency'] = 'GBP'
    df['Title'] = 'Grant to ' + df['Recipient Org:Name'].astype(str)
    df['Description'] = 'Grant to ' + df['Recipient Org:Name'].astype(str)
    df = df[['Identifier', 'Title', 'Description', 'Award Date', 'Currency', 'Amount Awarded', 'Amount Disbursed', 'IrrecoverableVAT', 'Classifications:0:Title', 'Classifications:1:Title', 'Grant Programme:Title', 'Planned Dates:Start Date', 'Funding Org:Identifier', 'Funding Org:Name', 'OrganisationCode', 'Recipient Org:Identifier', 'Recipient Org:Name', 'Last modified', 'Data Source']]
    df.to_csv(os.path.join('out','Leeds-City-Council-360Giving-'+fname),index=False)